# Jobathon Nov 2022 Time series Model

## Table of Contents
### 1. [Read Train and Test ](#read)
### 2. [Feature Generation](#feature)
### 3. [Train and Validation Split](#split)
### 4. [Model Evaluation using Facebook Prophet](#model_eval_fbprophet)
### 4. [Model Evaluation using Thyme Boost](#model_eval_thyme)
### 4. [Model Evaluation using Unobserved Components](#model_eval)
### 5. [Model Finalization for Test Prediction](#model_final)

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
pd.options.display.max_columns=500
pd.options.display.max_rows=500

In [4]:
from pandas.tseries.holiday import *

In [5]:
# !pip install pmdarima

In [6]:
# !pip install ThymeBoost

In [7]:
KAGGLE=False

In [8]:
if KAGGLE:
    path = '/kaggle/input/jobathon-nov-2022/'
else:
    path = 'input/'

<a id='read'></a>
## Read Train and Test Data

In [9]:
train=pd.read_csv(path+'train.csv')
print(train.shape)

(94992, 3)


In [10]:
test=pd.read_csv(path+'test.csv')
print(test.shape)

(26304, 2)


In [11]:
train.head()

,row_id,datetime,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500
3,4,2008-03-01 03:00:00,1034.742000
4,5,2008-03-01 04:00:00,1026.334500


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    94992 non-null  int64  
 1   datetime  94992 non-null  object 
 2   energy    93092 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [13]:
train['energy'].describe()
targetcol='energy'

In [14]:
train['datetime']=pd.to_datetime(train['datetime'],infer_datetime_format=True)
test['datetime']=pd.to_datetime(test['datetime'],infer_datetime_format=True)

In [15]:
train['datetime'].min(),train['datetime'].max()

(Timestamp('2008-03-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

<a id='feature'></a>
## Feature Generation

In [16]:
#create hour map based on business, non - business , sleeping hours etc.
hour_map={0:0,1:0,2:0,3:0,4:0,5:0,
         6:1,7:1,8:1,
         9:2,10:2,11:2,
         12:3,13:3,14:3,15:3,
         16:4,17:4,
         18:5,19:5,20:5,
         21:6,22:6,23:6}
 
#Monday and Sunday as group 1, Saturday as group 2, Otherdays as group 3
dayofweek_map = {0:1,6:1,
                5:2,
                1:3,2:3,3:3,4:3}

create basic date related features

In [17]:
def gen_datefeats(data):
    data['year']=data['datetime'].dt.year
    data['month']=data['datetime'].dt.month
    data['day']=data['datetime'].dt.day
    data['hour']=data['datetime'].dt.hour
    data['weekofyear']=data['datetime'].dt.isocalendar().week
    data['dayofweek']=data['datetime'].dt.dayofweek
    data['dayofweek_grp']=data['dayofweek'].replace(dayofweek_map)
    data['quarter']=data['datetime'].dt.quarter
    data['is_weekend']=data['datetime'].dt.dayofweek > 4
    data['day_part']=data['hour'].replace(hour_map)

In [18]:
gen_datefeats(train)
gen_datefeats(test)

create holiday features with special holiday denoting christmas long holidays

In [19]:
#generate holidays feature
def gen_holiday_feat(data,start,end):
    cal = USFederalHolidayCalendar()
    holiday_dates = cal.holidays(start=start, end=end)
    data['is_holiday'] = False
    mask = data['datetime'].dt.date.astype('datetime64').isin(holiday_dates)
    data.loc[mask,'is_holiday']=True   
    
    data['special_holiday']=False
    mask= ((data['datetime'].dt.month==12) & (data['datetime'].dt.day>=24))  \
           | ((data['datetime'].dt.month==1) & (data['datetime'].dt.day<3)) 
    data.loc[mask,'special_holiday']=True   
        
    return holiday_dates

In [20]:
holidays= gen_holiday_feat(train,train['datetime'].dt.date.min(),train['datetime'].dt.date.max())
print(holidays)
print(train['special_holiday'].value_counts())
train['is_holiday'].value_counts()

DatetimeIndex(['2008-05-26', '2008-07-04', '2008-09-01', '2008-10-13',
               '2008-11-11', '2008-11-27', '2008-12-25', '2009-01-01',
               '2009-01-19', '2009-02-16',
               ...
               '2018-01-01', '2018-01-15', '2018-02-19', '2018-05-28',
               '2018-07-04', '2018-09-03', '2018-10-08', '2018-11-12',
               '2018-11-22', '2018-12-25'],
              dtype='datetime64[ns]', length=107, freq=None)
False    92400
True      2592
Name: special_holiday, dtype: int64


False    92424
True      2568
Name: is_holiday, dtype: int64

In [21]:
train.head()

,row_id,datetime,energy,year,month,day,hour,weekofyear,dayofweek,dayofweek_grp,quarter,is_weekend,day_part,is_holiday,special_holiday
0,1,2008-03-01 00:00:00,1259.985563,2008,3,1,0,9,5,2,1,True,0,False,False
1,2,2008-03-01 01:00:00,1095.541500,2008,3,1,1,9,5,2,1,True,0,False,False
2,3,2008-03-01 02:00:00,1056.247500,2008,3,1,2,9,5,2,1,True,0,False,False
3,4,2008-03-01 03:00:00,1034.742000,2008,3,1,3,9,5,2,1,True,0,False,False
4,5,2008-03-01 04:00:00,1026.334500,2008,3,1,4,9,5,2,1,True,0,False,False


generate hour aggregate features month-wise, quarter-wise, week of year-wise and week day group wise

In [22]:
def gen_datetime_comb_feats(data):
    data['dayofweek_hr']=data['dayofweek'].astype('str') + '_'+data['hour'].astype('str')
    data['weekofyear_hr']=data['weekofyear'].astype('str') + '_'+data['hour'].astype('str')
    data['month_hr']=data['month'].astype('str') + '_'+data['hour'].astype('str')
    data['quarter_hr']=data['quarter'].astype('str') + '_'+data['hour'].astype('str')


def gen_mean_feats(train,test,cols,newcolname):
    grouped=train.groupby(cols)[targetcol].mean().reset_index()
    grouped.columns=cols+[newcolname]
    train=train.merge(grouped,on=cols)    
    test=test.merge(grouped,on=cols)   
    return train,test
    
def gen_mean_feats_all(train,test):
    train,test=gen_mean_feats(train,test,['month','hour'],'month_hour_mean')
    train,test=gen_mean_feats(train,test,['quarter','hour'],'quarter_hour_mean')
    train,test=gen_mean_feats(train,test,['weekofyear','hour'],'weekofyear_mean')
    train,test=gen_mean_feats(train,test,['dayofweek_grp','hour'],'dayofweek_grp_mean')  
    
    train.sort_values('datetime',inplace=True)
    train.reset_index(drop=True,inplace=True)
    test.sort_values('datetime',inplace=True)
    test.reset_index(drop=True,inplace=True)
    return train,test

In [23]:
train,test=gen_mean_feats_all(train,test)

In [24]:
gen_datetime_comb_feats(train)
gen_datetime_comb_feats(test)

In [25]:
train.head(10)

,row_id,datetime,energy,year,month,day,hour,weekofyear,dayofweek,dayofweek_grp,quarter,is_weekend,day_part,is_holiday,special_holiday,month_hour_mean,quarter_hour_mean,weekofyear_mean,dayofweek_grp_mean,dayofweek_hr,weekofyear_hr,month_hr,quarter_hr
0,1,2008-03-01 00:00:00,1259.985563,2008,3,1,0,9,5,2,1,True,0,False,False,1397.227508,1504.829986,1392.384333,1654.027936,5_0,9_0,3_0,1_0
1,2,2008-03-01 01:00:00,1095.541500,2008,3,1,1,9,5,2,1,True,0,False,False,1326.027668,1412.376876,1306.621744,1566.045064,5_1,9_1,3_1,1_1
2,3,2008-03-01 02:00:00,1056.247500,2008,3,1,2,9,5,2,1,True,0,False,False,1287.670058,1352.371501,1259.495448,1511.553768,5_2,9_2,3_2,1_2
3,4,2008-03-01 03:00:00,1034.742000,2008,3,1,3,9,5,2,1,True,0,False,False,1262.746667,1315.962924,1230.416947,1478.168590,5_3,9_3,3_3,1_3
4,5,2008-03-01 04:00:00,1026.334500,2008,3,1,4,9,5,2,1,True,0,False,False,1260.754616,1298.825422,1221.394201,1473.652068,5_4,9_4,3_4,1_4
5,6,2008-03-01 05:00:00,1033.768500,2008,3,1,5,9,5,2,1,True,0,False,False,1275.482187,1305.534563,1230.498917,1493.890931,5_5,9_5,3_5,1_5
6,7,2008-03-01 06:00:00,1086.780000,2008,3,1,6,9,5,2,1,True,1,False,False,1337.872697,1358.662339,1295.098037,1566.537865,5_6,9_6,3_6,1_6
7,8,2008-03-01 07:00:00,1211.742000,2008,3,1,7,9,5,2,1,True,1,False,False,1464.527791,1470.300187,1412.661900,1697.378282,5_7,9_7,3_7,1_7
8,9,2008-03-01 08:00:00,1293.693000,2008,3,1,8,9,5,2,1,True,1,False,False,1556.391201,1543.597804,1493.338281,1781.379368,5_8,9_8,3_8,1_8
9,10,2008-03-01 09:00:00,1318.915500,2008,3,1,9,9,5,2,1,True,2,False,False,1577.528044,1589.814926,1517.786396,1804.679326,5_9,9_9,3_9,1_9


In [26]:
test.head()

,row_id,datetime,year,month,day,hour,weekofyear,dayofweek,dayofweek_grp,quarter,is_weekend,day_part,month_hour_mean,quarter_hour_mean,weekofyear_mean,dayofweek_grp_mean,dayofweek_hr,weekofyear_hr,month_hr,quarter_hr
0,94993,2019-01-01 00:00:00,2019,1,1,0,1,1,3,1,False,0,1661.002682,1504.829986,1711.103516,1625.815769,1_0,1_0,1_0,1_0
1,94994,2019-01-01 01:00:00,2019,1,1,1,1,1,3,1,False,0,1543.596544,1412.376876,1592.733254,1543.548695,1_1,1_1,1_1,1_1
2,94995,2019-01-01 02:00:00,2019,1,1,2,1,1,3,1,False,0,1462.390875,1352.371501,1506.964956,1493.577402,1_2,1_2,1_2,1_2
3,94996,2019-01-01 03:00:00,2019,1,1,3,1,1,3,1,False,0,1410.076914,1315.962924,1439.720612,1465.825460,1_3,1_3,1_3,1_3
4,94997,2019-01-01 04:00:00,2019,1,1,4,1,1,3,1,False,0,1378.289974,1298.825422,1415.453093,1459.997669,1_4,1_4,1_4,1_4


In [27]:
train['year'].unique()

array([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

In [28]:
test['datetime'].min(),test['datetime'].max()

(Timestamp('2019-01-01 00:00:00'), Timestamp('2021-12-31 23:00:00'))

In [29]:
holidays= gen_holiday_feat(test,test['datetime'].dt.date.min(),test['datetime'].dt.date.max())
print(len(holidays))
print(holidays)
print(test['special_holiday'].value_counts())
test['is_holiday'].value_counts()

31
DatetimeIndex(['2019-01-01', '2019-01-21', '2019-02-18', '2019-05-27',
               '2019-07-04', '2019-09-02', '2019-10-14', '2019-11-11',
               '2019-11-28', '2019-12-25', '2020-01-01', '2020-01-20',
               '2020-02-17', '2020-05-25', '2020-07-03', '2020-09-07',
               '2020-10-12', '2020-11-11', '2020-11-26', '2020-12-25',
               '2021-01-01', '2021-01-18', '2021-02-15', '2021-05-31',
               '2021-07-05', '2021-09-06', '2021-10-11', '2021-11-11',
               '2021-11-25', '2021-12-24', '2021-12-31'],
              dtype='datetime64[ns]', freq=None)
False    25584
True       720
Name: special_holiday, dtype: int64


False    25560
True       744
Name: is_holiday, dtype: int64

In [30]:
targetcol = 'energy'

In [31]:
# train[targetcol].fillna(train[targetcol].mean(),inplace=True)
train[targetcol].fillna(method='ffill',inplace=True)

Create Lag Features

In [32]:
def create_lag(data,lagno_list):
    res = pd.DataFrame()
    for i in lagno_list:
        shifted = data.shift(i)
        res=pd.concat([res,shifted],axis=1)

#     res=pd.concat([data.shift(i) for i in lagno_list],axis=1)
    res.columns=[f'lag_{i}' for i in lagno_list]
    return res

In [33]:
test.shape

(26304, 22)

In [34]:
train['istrain']=1
test['istrain']=0
combined = pd.concat([train,test],axis=0) 
#lag 1 year, 3 year, quarter, month,week
lag_df = create_lag(combined[targetcol],[24,168,720,2160,8760,26304])
combined=pd.concat([combined,lag_df],axis=1)
train=combined[combined['istrain']==1]
test=combined[combined['istrain']==0]

del combined,train['istrain'],test['istrain'],test[targetcol]
print(train.shape,test.shape)

(94992, 29) (26304, 28)


In [35]:
train.head()

,row_id,datetime,energy,year,month,day,hour,weekofyear,dayofweek,dayofweek_grp,quarter,is_weekend,day_part,is_holiday,special_holiday,month_hour_mean,quarter_hour_mean,weekofyear_mean,dayofweek_grp_mean,dayofweek_hr,weekofyear_hr,month_hr,quarter_hr,lag_24,lag_168,lag_720,lag_2160,lag_8760,lag_26304
0,1,2008-03-01 00:00:00,1259.985563,2008,3,1,0,9,5,2,1,True,0,False,False,1397.227508,1504.829986,1392.384333,1654.027936,5_0,9_0,3_0,1_0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2008-03-01 01:00:00,1095.541500,2008,3,1,1,9,5,2,1,True,0,False,False,1326.027668,1412.376876,1306.621744,1566.045064,5_1,9_1,3_1,1_1,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2008-03-01 02:00:00,1056.247500,2008,3,1,2,9,5,2,1,True,0,False,False,1287.670058,1352.371501,1259.495448,1511.553768,5_2,9_2,3_2,1_2,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2008-03-01 03:00:00,1034.742000,2008,3,1,3,9,5,2,1,True,0,False,False,1262.746667,1315.962924,1230.416947,1478.168590,5_3,9_3,3_3,1_3,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2008-03-01 04:00:00,1026.334500,2008,3,1,4,9,5,2,1,True,0,False,False,1260.754616,1298.825422,1221.394201,1473.652068,5_4,9_4,3_4,1_4,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
train.columns

Index(['row_id', 'datetime', 'energy', 'year', 'month', 'day', 'hour',
       'weekofyear', 'dayofweek', 'dayofweek_grp', 'quarter', 'is_weekend',
       'day_part', 'is_holiday', 'special_holiday', 'month_hour_mean',
       'quarter_hour_mean', 'weekofyear_mean', 'dayofweek_grp_mean',
       'dayofweek_hr', 'weekofyear_hr', 'month_hr', 'quarter_hr', 'lag_24',
       'lag_168', 'lag_720', 'lag_2160', 'lag_8760', 'lag_26304'],
      dtype='object')

Fill null values in train data using previous hour values

In [37]:
cols = [col for col in train.columns if col.startswith('lag_')]
target_mean = train[targetcol].mean()
for col in cols:
    train[col].fillna(0,inplace=True)

<a id='split'></a>
## Train and Validation Split

Validation Set from 2016 to 2018 <br>
Train Set from 2008 to 2015

In [38]:
import datetime 
train_start = datetime.datetime(year=2008,month=1,day=1,hour=0)
val_start = datetime.datetime(year=2016,month=1,day=1,hour=0)
val_end = datetime.datetime(year=2018,month=12,day=31,hour=23)

X_val= train[(train['datetime']>=val_start) & (train['datetime']<=val_end)].copy()
X_train= train[(train['datetime']>=train_start) & (train['datetime']<val_start)].copy()
print(X_train.shape)
print(X_val.shape)
X_val.head()           

(68688, 29)
(26304, 29)


,row_id,datetime,energy,year,month,day,hour,weekofyear,dayofweek,dayofweek_grp,quarter,is_weekend,day_part,is_holiday,special_holiday,month_hour_mean,quarter_hour_mean,weekofyear_mean,dayofweek_grp_mean,dayofweek_hr,weekofyear_hr,month_hr,quarter_hr,lag_24,lag_168,lag_720,lag_2160,lag_8760,lag_26304
68688,68689,2016-01-01 00:00:00,2002.2470,2016,1,1,0,53,4,3,1,False,0,True,True,1661.002682,1504.829986,1874.91005,1625.815769,4_0,53_0,1_0,1_0,2045.0170,2274.7530,1983.0616,1878.0918,2287.2788,1479.1658
68689,68690,2016-01-01 01:00:00,1843.3870,2016,1,1,1,53,4,3,1,False,0,True,True,1543.596544,1412.376876,1734.62185,1543.548695,4_1,53_1,1_1,1_1,1881.1468,2118.4592,1827.3788,1878.0918,2120.5124,1375.2627
68690,68691,2016-01-01 02:00:00,1735.1178,2016,1,1,2,53,4,3,1,False,0,True,True,1462.390875,1352.371501,1633.81205,1493.577402,4_2,53_2,1_2,1_2,1781.7982,2020.4548,1719.9650,1627.7040,2015.9704,1307.8261
68691,68692,2016-01-01 03:00:00,1666.1970,2016,1,1,3,53,4,3,1,False,0,True,True,1410.076914,1315.962924,1578.16560,1465.825460,4_3,53_3,1_3,1_3,1719.5984,1936.5034,1648.2336,1564.5266,1939.9740,1261.5295
68692,68693,2016-01-01 04:00:00,1630.3924,2016,1,1,4,53,4,3,1,False,0,True,True,1378.289974,1298.825422,1542.43910,1459.997669,4_4,53_4,1_4,1_4,1657.1542,1895.8108,1603.2640,1541.5530,1885.3868,1228.8682


In [39]:
val_target = X_val[targetcol]
print(X_val[targetcol].isnull().sum())

0


Create simple validation prediction baseline using train mean value 

In [40]:
from sklearn.metrics import mean_squared_error

In [41]:
#compute baseline error by predicting train energy mean as the energy for all time
val_preds_baseline = np.full(len(X_val),train[targetcol].mean())

In [42]:
#compute error score on baseline predictions
val_score = mean_squared_error(val_target,val_preds_baseline,squared=False)
print('valid score:',val_score)

valid score: 431.7300102305795


<a id='model_eval_fbprophet'></a>
## Model Evaluation using Facebook Prophet

In [43]:
# !pip install pystan==2.19.1.1

In [44]:
# !pip install fbprophet

In [45]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
from fbprophet.plot import plot_plotly
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [46]:
from multiprocessing import cpu_count
from joblib import Parallel, delayed
from multiprocessing import cpu_count

In [47]:
from sklearn.model_selection import ParameterGrid

In [48]:
def tune_prophet_params(param):
    val_start   = 68688
    val_period  = 26304
    print(param)
    np.random.seed(0)
    train_model =Prophet(uncertainty_samples=0,
                        changepoint_prior_scale = param['changepoint_prior_scale'],
#                              n_changepoints = param['n_changepoints'],
                         changepoint_range = param['changepoint_range'],
                         weekly_seasonality=True,
                         daily_seasonality = True,
                         yearly_seasonality = True,
                         interval_width=0.95)
    train_model.fit(train_df[:val_start])
    future_df = train_model.make_future_dataframe(periods=val_period, freq='H',include_history = False)
#     future_df['cap']=4000
#     future_df['floor']=1000
    train_forecast = train_model.predict(future_df)
    val=train_forecast[['ds','yhat']]
    Actual = train_df[val_start:val_start+val_period]
    RMSE = mean_squared_error(Actual['y'],abs(val['yhat']),squared=False)
    print('RMSE------------------------------------',RMSE)
    model_param = {'RMSE':RMSE}
    model_param.update(param)

    return model_param

Evaluation of Best Tuned Model with additional regressors

In [49]:
def add_regressors(data_prophet,data_orig,regressors):
    df_with_reg = pd.concat([data_prophet.reset_index(drop=True),
                             data_orig[regressors].head(len(data_prophet)).reset_index(drop=True)],axis=1)
    return df_with_reg

In [50]:
# cols = ['hour','dayofweek','weekofyear','quarter_hr','dayofweek_hr','month_hr']
cols = ['hour','dayofweek','quarter','quarter_hr','month_hr']
exog_train = pd.get_dummies(X_train.set_index('datetime')[cols],columns=cols,prefix=cols)
exog_test = pd.get_dummies(X_val.set_index('datetime')[cols],columns=cols,prefix=cols)

y_train = X_train.set_index('datetime')[targetcol].copy()
y_test = X_val.set_index('datetime')[targetcol].copy()

In [51]:
# regressors = list(exog_train.columns)
regressors = [col for col in exog_train.columns if col.startswith('hour_')]
X_train_df = pd.concat([exog_train,y_train],axis=1)
X_train_df = X_train_df.reset_index().rename(columns={'datetime': 'ds', 
                        targetcol: 'y'})
print(X_train_df.shape)
X_train_df.head()

(68688, 421)


,ds,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,quarter_1,quarter_2,quarter_3,quarter_4,quarter_hr_1_0,quarter_hr_1_1,quarter_hr_1_10,quarter_hr_1_11,quarter_hr_1_12,quarter_hr_1_13,quarter_hr_1_14,quarter_hr_1_15,quarter_hr_1_16,quarter_hr_1_17,quarter_hr_1_18,quarter_hr_1_19,quarter_hr_1_2,quarter_hr_1_20,quarter_hr_1_21,quarter_hr_1_22,quarter_hr_1_23,quarter_hr_1_3,quarter_hr_1_4,quarter_hr_1_5,quarter_hr_1_6,quarter_hr_1_7,quarter_hr_1_8,quarter_hr_1_9,quarter_hr_2_0,quarter_hr_2_1,quarter_hr_2_10,quarter_hr_2_11,quarter_hr_2_12,quarter_hr_2_13,quarter_hr_2_14,quarter_hr_2_15,quarter_hr_2_16,quarter_hr_2_17,quarter_hr_2_18,quarter_hr_2_19,quarter_hr_2_2,quarter_hr_2_20,quarter_hr_2_21,quarter_hr_2_22,quarter_hr_2_23,quarter_hr_2_3,quarter_hr_2_4,quarter_hr_2_5,quarter_hr_2_6,quarter_hr_2_7,quarter_hr_2_8,quarter_hr_2_9,quarter_hr_3_0,quarter_hr_3_1,quarter_hr_3_10,quarter_hr_3_11,quarter_hr_3_12,quarter_hr_3_13,quarter_hr_3_14,quarter_hr_3_15,quarter_hr_3_16,quarter_hr_3_17,quarter_hr_3_18,quarter_hr_3_19,quarter_hr_3_2,quarter_hr_3_20,quarter_hr_3_21,quarter_hr_3_22,quarter_hr_3_23,quarter_hr_3_3,quarter_hr_3_4,quarter_hr_3_5,quarter_hr_3_6,quarter_hr_3_7,quarter_hr_3_8,quarter_hr_3_9,quarter_hr_4_0,quarter_hr_4_1,quarter_hr_4_10,quarter_hr_4_11,quarter_hr_4_12,quarter_hr_4_13,quarter_hr_4_14,quarter_hr_4_15,quarter_hr_4_16,quarter_hr_4_17,quarter_hr_4_18,quarter_hr_4_19,quarter_hr_4_2,quarter_hr_4_20,quarter_hr_4_21,quarter_hr_4_22,quarter_hr_4_23,quarter_hr_4_3,quarter_hr_4_4,quarter_hr_4_5,quarter_hr_4_6,quarter_hr_4_7,quarter_hr_4_8,quarter_hr_4_9,month_hr_10_0,month_hr_10_1,month_hr_10_10,month_hr_10_11,month_hr_10_12,month_hr_10_13,month_hr_10_14,month_hr_10_15,month_hr_10_16,month_hr_10_17,month_hr_10_18,month_hr_10_19,month_hr_10_2,month_hr_10_20,month_hr_10_21,month_hr_10_22,month_hr_10_23,month_hr_10_3,month_hr_10_4,month_hr_10_5,month_hr_10_6,month_hr_10_7,month_hr_10_8,month_hr_10_9,month_hr_11_0,month_hr_11_1,month_hr_11_10,month_hr_11_11,month_hr_11_12,month_hr_11_13,month_hr_11_14,month_hr_11_15,month_hr_11_16,month_hr_11_17,month_hr_11_18,month_hr_11_19,month_hr_11_2,month_hr_11_20,month_hr_11_21,month_hr_11_22,month_hr_11_23,month_hr_11_3,month_hr_11_4,month_hr_11_5,month_hr_11_6,month_hr_11_7,month_hr_11_8,month_hr_11_9,month_hr_12_0,month_hr_12_1,month_hr_12_10,month_hr_12_11,month_hr_12_12,month_hr_12_13,month_hr_12_14,month_hr_12_15,month_hr_12_16,month_hr_12_17,month_hr_12_18,month_hr_12_19,month_hr_12_2,month_hr_12_20,month_hr_12_21,month_hr_12_22,month_hr_12_23,month_hr_12_3,month_hr_12_4,month_hr_12_5,month_hr_12_6,month_hr_12_7,month_hr_12_8,month_hr_12_9,month_hr_1_0,month_hr_1_1,month_hr_1_10,month_hr_1_11,month_hr_1_12,month_hr_1_13,month_hr_1_14,month_hr_1_15,month_hr_1_16,month_hr_1_17,month_hr_1_18,month_hr_1_19,month_hr_1_2,month_hr_1_20,month_hr_1_21,month_hr_1_22,month_hr_1_23,month_hr_1_3,month_hr_1_4,month_hr_1_5,month_hr_1_6,month_hr_1_7,month_hr_1_8,month_hr_1_9,month_hr_2_0,month_hr_2_1,month_hr_2_10,month_hr_2_11,month_hr_2_12,month_hr_2_13,month_hr_2_14,month_hr_2_15,month_hr_2_16,month_hr_2_17,month_hr_2_18,month_hr_2_19,month_hr_2_2,month_hr_2_20,month_hr_2_21,month_hr_2_22,month_hr_2_23,month_hr_2_3,month_hr_2_4,month_hr_2_5,month_hr_2_6,month_hr_2_7,month_hr_2_8,month_hr_2_9,month_hr_3_0,month_hr_3_1,month_hr_3_10,month_hr_3_11,month_hr_3_12,month_hr_3_13,month_hr_3_14,month_hr_3_15,month_hr_3_16,month_hr_3_17,month_hr_3_18,month_hr_3_19,month_hr_3_2,month_hr_3_20,month_hr_3_21,month_hr_3_22,month_hr_3_23,month_hr_3_3,month_hr_3_4,month_hr_3_5,month_hr_3_6,month_hr_3_7,month_hr_3_8,month_hr_3_9,month_hr_4_0,month_hr_4_1,month_hr_4_10,month_hr_4_11,month_hr_4_12,month_hr_4_13,month_hr_4_14,month_hr_4_15,month_hr_4_16,month_hr_4_17,month_hr_4_18,month_hr_4_19,

In [52]:
X_val_df = pd.concat([exog_test,y_test],axis=1)
X_val_df = X_val_df.reset_index().rename(columns={'datetime': 'ds', 
                        targetcol: 'y'})
print(X_val_df.shape)
X_val_df.head()

(26304, 421)


,ds,hour_0,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,quarter_1,quarter_2,quarter_3,quarter_4,quarter_hr_1_0,quarter_hr_1_1,quarter_hr_1_10,quarter_hr_1_11,quarter_hr_1_12,quarter_hr_1_13,quarter_hr_1_14,quarter_hr_1_15,quarter_hr_1_16,quarter_hr_1_17,quarter_hr_1_18,quarter_hr_1_19,quarter_hr_1_2,quarter_hr_1_20,quarter_hr_1_21,quarter_hr_1_22,quarter_hr_1_23,quarter_hr_1_3,quarter_hr_1_4,quarter_hr_1_5,quarter_hr_1_6,quarter_hr_1_7,quarter_hr_1_8,quarter_hr_1_9,quarter_hr_2_0,quarter_hr_2_1,quarter_hr_2_10,quarter_hr_2_11,quarter_hr_2_12,quarter_hr_2_13,quarter_hr_2_14,quarter_hr_2_15,quarter_hr_2_16,quarter_hr_2_17,quarter_hr_2_18,quarter_hr_2_19,quarter_hr_2_2,quarter_hr_2_20,quarter_hr_2_21,quarter_hr_2_22,quarter_hr_2_23,quarter_hr_2_3,quarter_hr_2_4,quarter_hr_2_5,quarter_hr_2_6,quarter_hr_2_7,quarter_hr_2_8,quarter_hr_2_9,quarter_hr_3_0,quarter_hr_3_1,quarter_hr_3_10,quarter_hr_3_11,quarter_hr_3_12,quarter_hr_3_13,quarter_hr_3_14,quarter_hr_3_15,quarter_hr_3_16,quarter_hr_3_17,quarter_hr_3_18,quarter_hr_3_19,quarter_hr_3_2,quarter_hr_3_20,quarter_hr_3_21,quarter_hr_3_22,quarter_hr_3_23,quarter_hr_3_3,quarter_hr_3_4,quarter_hr_3_5,quarter_hr_3_6,quarter_hr_3_7,quarter_hr_3_8,quarter_hr_3_9,quarter_hr_4_0,quarter_hr_4_1,quarter_hr_4_10,quarter_hr_4_11,quarter_hr_4_12,quarter_hr_4_13,quarter_hr_4_14,quarter_hr_4_15,quarter_hr_4_16,quarter_hr_4_17,quarter_hr_4_18,quarter_hr_4_19,quarter_hr_4_2,quarter_hr_4_20,quarter_hr_4_21,quarter_hr_4_22,quarter_hr_4_23,quarter_hr_4_3,quarter_hr_4_4,quarter_hr_4_5,quarter_hr_4_6,quarter_hr_4_7,quarter_hr_4_8,quarter_hr_4_9,month_hr_10_0,month_hr_10_1,month_hr_10_10,month_hr_10_11,month_hr_10_12,month_hr_10_13,month_hr_10_14,month_hr_10_15,month_hr_10_16,month_hr_10_17,month_hr_10_18,month_hr_10_19,month_hr_10_2,month_hr_10_20,month_hr_10_21,month_hr_10_22,month_hr_10_23,month_hr_10_3,month_hr_10_4,month_hr_10_5,month_hr_10_6,month_hr_10_7,month_hr_10_8,month_hr_10_9,month_hr_11_0,month_hr_11_1,month_hr_11_10,month_hr_11_11,month_hr_11_12,month_hr_11_13,month_hr_11_14,month_hr_11_15,month_hr_11_16,month_hr_11_17,month_hr_11_18,month_hr_11_19,month_hr_11_2,month_hr_11_20,month_hr_11_21,month_hr_11_22,month_hr_11_23,month_hr_11_3,month_hr_11_4,month_hr_11_5,month_hr_11_6,month_hr_11_7,month_hr_11_8,month_hr_11_9,month_hr_12_0,month_hr_12_1,month_hr_12_10,month_hr_12_11,month_hr_12_12,month_hr_12_13,month_hr_12_14,month_hr_12_15,month_hr_12_16,month_hr_12_17,month_hr_12_18,month_hr_12_19,month_hr_12_2,month_hr_12_20,month_hr_12_21,month_hr_12_22,month_hr_12_23,month_hr_12_3,month_hr_12_4,month_hr_12_5,month_hr_12_6,month_hr_12_7,month_hr_12_8,month_hr_12_9,month_hr_1_0,month_hr_1_1,month_hr_1_10,month_hr_1_11,month_hr_1_12,month_hr_1_13,month_hr_1_14,month_hr_1_15,month_hr_1_16,month_hr_1_17,month_hr_1_18,month_hr_1_19,month_hr_1_2,month_hr_1_20,month_hr_1_21,month_hr_1_22,month_hr_1_23,month_hr_1_3,month_hr_1_4,month_hr_1_5,month_hr_1_6,month_hr_1_7,month_hr_1_8,month_hr_1_9,month_hr_2_0,month_hr_2_1,month_hr_2_10,month_hr_2_11,month_hr_2_12,month_hr_2_13,month_hr_2_14,month_hr_2_15,month_hr_2_16,month_hr_2_17,month_hr_2_18,month_hr_2_19,month_hr_2_2,month_hr_2_20,month_hr_2_21,month_hr_2_22,month_hr_2_23,month_hr_2_3,month_hr_2_4,month_hr_2_5,month_hr_2_6,month_hr_2_7,month_hr_2_8,month_hr_2_9,month_hr_3_0,month_hr_3_1,month_hr_3_10,month_hr_3_11,month_hr_3_12,month_hr_3_13,month_hr_3_14,month_hr_3_15,month_hr_3_16,month_hr_3_17,month_hr_3_18,month_hr_3_19,month_hr_3_2,month_hr_3_20,month_hr_3_21,month_hr_3_22,month_hr_3_23,month_hr_3_3,month_hr_3_4,month_hr_3_5,month_hr_3_6,month_hr_3_7,month_hr_3_8,month_hr_3_9,month_hr_4_0,month_hr_4_1,month_hr_4_10,month_hr_4_11,month_hr_4_12,month_hr_4_13,month_hr_4_14,month_hr_4_15,month_hr_4_16,month_hr_4_17,month_hr_4_18,month_hr_4_19,

In [53]:
import random
seed=100

In [54]:
# X_train_df['cap']=4000
# X_train_df['floor']=1000
# X_val_df['cap']=4000
# X_val_df['floor']=1000

In [74]:
%%time
val_start   = len(X_train_df)#10000
val_period  = len(X_val_df)#48
# val_start   = 68688
# val_period  = 26304
np.random.seed(seed)

train_model =Prophet(
                    uncertainty_samples=0,
#                      mcmc_samples = 100,
                    changepoint_prior_scale =0.01,
#                     n_changepoints = 50,
                     changepoint_range = 0.0,
                     weekly_seasonality=True,
                     daily_seasonality = True,
                     yearly_seasonality =True,
                     interval_width=0.95)

# train_model.add_regressor('hour_1',
#                               prior_scale=10,#100
#                               standardize=False
#                              )

# for col in regressors:
#     train_model.add_regressor(col,
#                               prior_scale=10,#100
#                               standardize=False
#                              )


# train_mask = X_train_df[f'hour_{i}']==1
# val_mask = X_val_df[f'hour_{i}']==1
train_model.fit(X_train_df[:val_start],
#                 seed=seed
               )
print('Fit completed')

train_forecast_orig = train_model.predict(X_val_df[:val_period])
# RMSE = mean_squared_error(X_val_df[:val_period]['y'],abs(train_forecast['yhat']),squared=False)
RMSE = mean_squared_error(X_val_df[:val_period]['y'],train_forecast_orig['yhat'],squared=False)
# RMSE = mean_squared_error(X_val_df['y'],abs(train_forecast['yhat']),squared=False)
print('RMSE------------------------------------',RMSE)

INFO:fbprophet:n_changepoints greater than number of observations. Using -1.


Initial log joint probability = -366.669
Fit completed
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      30        161107   0.000189219        1359.2   3.416e-08       0.001       95  LS failed, Hessian reset 
      47        161109   5.98541e-05        443.65   1.029e-07       0.001      159  LS failed, Hessian reset 
      60        161109   5.39922e-05       125.347           1           1      180   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance
RMSE------------------------------------ 224.36625209548467
CPU times: user 5.12 s, sys: 85.8 ms, total: 5.2 s
Wall time: 5.14 s


In [56]:
# train_model.fit(X_train_df[train_mask][:val_start],
# #                 seed=seed
#                )
# print('Fit completed')

# train_forecast = train_model.predict(X_val_df[val_mask][:val_period])
# # RMSE = mean_squared_error(X_val_df[:val_period]['y'],abs(train_forecast['yhat']),squared=False)
# RMSE = mean_squared_error(X_val_df[val_mask][:val_period]['y'],train_forecast['yhat'],squared=False)
# # RMSE = mean_squared_error(X_val_df['y'],abs(train_forecast['yhat']),squared=False)
# print('RMSE------------------------------------',RMSE)

In [57]:
#default   hour8    non hour8
# 224.36   286.32   221.27

# 208.65   224.99
# 212.93   224.84

# hour =1  186.346 (orig: 208.6)


In [75]:
rmses=[]
for i in range(24):
    mask = X_val_df[f'hour_{i}']==1
    RMSE = mean_squared_error(X_val_df[mask][:val_period]['y'],train_forecast_orig[mask]['yhat'],squared=False)
#     print(f'hour:{i}  RMSE:{RMSE}')
    rmses.append(RMSE)

hour_rmses = pd.DataFrame()    
hour_rmses['hour'] = range(24)
hour_rmses['rmse'] = rmses
hour_rmses

,hour,rmse
0,0,199.293125
1,1,212.937914
2,2,222.226774
3,3,231.289022
4,4,242.293174
5,5,255.752924
6,6,263.742334
7,7,282.285694
8,8,286.326049
9,9,254.876139


In [ ]:
mask = X_val_df['hour_1']==1
RMSE = mean_squared_error(X_val_df[mask][:val_period]['y'],train_forecast[mask]['yhat'],squared=False)
print('RMSE------------------------------------',RMSE)

In [ ]:
mask = X_val_df['hour_1']==0
RMSE = mean_squared_error(X_val_df[mask][:val_period]['y'],train_forecast[mask]['yhat'],squared=False)
print('RMSE------------------------------------',RMSE)

In [ ]:
y_mean = (train_forecast['yhat_upper'] + train_forecast['yhat_lower'] ) / 2
ydiff =  y_mean - train_forecast['yhat']
print(ydiff.describe())
RMSE = mean_squared_error(X_val_df[:val_period]['y'],y_mean,squared=False)
print('RMSE------------------------------------',RMSE)

In [ ]:
# 2579.996822662037

In [ ]:
# train_model.add_seasonality(name='daily', period=24, fourier_order=15, prior_scale=0.1)
# train_model.add_seasonality(name='weekly', period=168, fourier_order=3, prior_scale=0.1)
# train_model.add_seasonality(name='yearly', period=8760, fourier_order=3, prior_scale=0.01)

# RMSE: 683 SCALE: 50,15,2 ORDEr: 15,3,3
# RMSE: 659 SCALE: 15,10,2 ORDEr: 15,3,3
# RMSE: 445 SCALE: 1,1,0.1 ORDER: 15,3,3
# RMSE: 440 SCALE: 0.1,0.1,0.01 ORDER: 15,3,3

# RMSE: 250 mcmc_samples=50, adapt_delta: 0.85
# RMSE: 245 mcmc_samples=50, adapt_delta: 0.99
# RMSE: 227 mcmc_samples=50, adapt_delta: 0.99 , chains=1 (time: 4 min)
# RMSE: 225 mcmc_samples=50, adapt_delta: 0.99 , chains=1 with hour regressors (time: 4 min)
# RMSE: 225 changepoint_prior_scale=0.1, mcmc_samples=50, adapt_delta: 0.99 , chains=1 (time: 1 min)
# RMSE: 222 changepoint_prior_scale=0.5, mcmc_samples=50, adapt_delta: 0.99 , chains=1 (time: 1 min)


# RMSE: 284 mcmc_samples=100, adapt_delta: 0.99 , chains=2 (time: 30 min)

# train_w_reg = add_regressors(train_df[:val_start],X_train,regressors)

In [59]:
%%time

# ************** IMPORTANT NOTE *************************
# changepoint_prior_scale should be set to 0.1, otherwise the code hangs for hour=8 

val_start   = len(X_train_df)#10000
val_period  = len(X_val_df)#48
np.random.seed(seed)

preds = np.zeros(len(X_val_df))

rmses=[]

for i in range(24):
    
    print()
    print(f'*********** HOUR {i} *******************')
    
#     if i==8:
#         rmses.append(0)
#         continue


    train_model =Prophet(
                        uncertainty_samples=0,
                        changepoint_prior_scale =0.1,
                         changepoint_range = 0.0,
                         weekly_seasonality=True,
                         daily_seasonality = False,
                         yearly_seasonality =True,
                         interval_width=0.95)


    # for col in regressors:
    #     train_model.add_regressor(col,
    #                               prior_scale=10,#100
    #                               standardize=False
    #                              )

    train_mask = X_train_df[f'hour_{i}']==1
    val_mask = X_val_df[f'hour_{i}']==1
    train_model.fit(X_train_df[train_mask][:val_start],
    #                 seed=seed
                   )
    print('Fit completed')

    train_forecast = train_model.predict(X_val_df[val_mask][:val_period])
    RMSE = mean_squared_error(X_val_df[val_mask][:val_period]['y'],train_forecast['yhat'],squared=False)
    print(f'hour:{i}  RMSE:{RMSE}')
    # print('RMSE------------------------------------',RMSE)
    
    preds[val_mask]=train_forecast['yhat']

    rmses.append(RMSE)

hour_rmses_pred = pd.DataFrame()    
hour_rmses_pred['hour'] = range(24)
hour_rmses_pred['rmse'] = rmses
hour_rmses_pred

INFO:fbprophet:n_changepoints greater than number of observations. Using -1.



*********** HOUR 0 *******************


Initial log joint probability = -16.0677
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      22       6761.81   0.000230969       143.551   1.327e-06       0.001       60  LS failed, Hessian reset 
      40       6761.97   0.000292458        69.776    7.21e-06       0.001      121  LS failed, Hessian reset 
      55       6761.99   2.18367e-05       10.6126   1.902e-06       0.001      178  LS failed, Hessian reset 
      58       6761.99   3.57465e-06       8.90171   7.032e-08       0.001      237  LS failed, Hessian reset 
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made
Fit completed


INFO:fbprophet:n_changepoints greater than number of observations. Using -1.



Initial log joint probability = -2733.5
Iteration  1. Log joint probability =    1360.75. Improved by 4094.26.
Iteration  2. Log joint probability =     5752.2. Improved by 4391.44.
Iteration  3. Log joint probability =     6262.3. Improved by 510.098.
Iteration  4. Log joint probability =    6275.19. Improved by 12.8909.
Iteration  5. Log joint probability =    6382.75. Improved by 107.563.
Iteration  6. Log joint probability =    6704.72. Improved by 321.971.
Iteration  7. Log joint probability =    6713.19. Improved by 8.46583.
Iteration  8. Log joint probability =    6725.26. Improved by 12.0718.
Iteration  9. Log joint probability =    6728.02. Improved by 2.75894.
Iteration 10. Log joint probability =    6728.92. Improved by 0.900863.
Iteration 11. Log joint probability =    6728.93. Improved by 0.0166224.
Iteration 12. Log joint probability =    6729.07. Improved by 0.140848.
Iteration 13. Log joint probability =    6761.77. Improved by 32.6947.
Iteration 14. Log joint probabil

...

    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      28       6891.05   0.000426436       173.379   7.692e-07       0.001       82  LS failed, Hessian reset 
      46       6891.29    0.00021719       118.512   1.422e-06       0.001      142  LS failed, Hessian reset 
      63       6891.32   7.45439e-06        8.5124   1.138e-07       0.001      222  LS failed, Hessian reset 
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -2732.8
Iteration  1. Log joint probability =    2224.46. Improved by 4957.26.
Iteration  2. Log joint probability =    5713.63. Improved by 3489.18.
Iteration  3. Log joint probability =     6470.5. Improved by 756.863.
Iteration  4. Log joint probability =    6775.67. Improved by 305.174.
Iteration  5. Log joint probability =    6781.73. Improved by 6.06201.
Iteration  6. Log joint probability =    6786.09. Improved

INFO:fbprophet:n_changepoints greater than number of observations. Using -1.


hour:22  RMSE:195.22111442240978

*********** HOUR 23 *******************


Initial log joint probability = -14.3859
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      26       6857.59   0.000347668       213.718    9.85e-07       0.001       75  LS failed, Hessian reset 
      42       6857.87   4.50424e-05       24.4441   9.216e-07       0.001      135  LS failed, Hessian reset 
      57       6857.87   1.87783e-05       8.94946   5.988e-08       0.001      212  LS failed, Hessian reset 
Optimization terminated with error: 
  Line search failed to achieve a sufficient decrease, no more progress can be made

Initial log joint probability = -2731.82
Iteration  1. Log joint probability =    3531.45. Improved by 6263.27.
Iteration  2. Log joint probability =    5240.65. Improved by 1709.21.
Iteration  3. Log joint probability =    5799.46. Improved by 558.805.
Iteration  4. Log joint probability =    6562.82. Improved by 763.36.
Iteration  5. Log joint probability =    6568.28. Improved by 5.45488.
Iteration  6. Log

,hour,rmse
0,0,187.288632
1,1,186.364082
2,2,187.472083
3,3,188.145401
4,4,189.297248
5,5,191.895015
6,6,194.276030
7,7,199.528927
8,8,198.445809
9,9,193.800730


In [60]:
# val_mask_8 = X_val_df['hour_8']==1
# preds[val_mask_8] = train_forecast_orig[val_mask_8]['yhat']

In [65]:
pd.Series(preds).describe()

count    26304.000000
mean      2021.803862
std        256.990237
min       1370.213967
25%       1836.925619
50%       2024.884824
75%       2204.027094
max       2730.205304
dtype: float64

In [62]:
train_forecast = train_model.predict(X_val_df[:val_period])
RMSE = mean_squared_error(X_val_df[:val_period]['y'],preds,squared=False)
print('RMSE------------------------------------',RMSE)

RMSE------------------------------------ 196.3774674833253


In [111]:
# for col in regressors:
#     train_model.add_regressor(col,prior_scale=2,standardize=False)

In [112]:
# %%time
# np.random.seed(100)
# train_model.fit(X_train_df[:val_start],
#                 seed=100,
#                 chains=4,
#                 control={'adapt_delta': 0.99,
# #                          'max_treedepth': 20
#                         })

In [63]:
# future_df = train_model.make_future_dataframe(periods=val_period, freq='H',include_history = False)
# X_val_df['cap']=4000
# X_val_df['floor']=1000
# future_w_reg = add_regressors(future_df,X_val,regressors)
# np.random.seed(seed)
# train_forecast = train_model.predict(X_val_df[:val_period])
# RMSE = mean_squared_error(X_val_df[:val_period]['y'],abs(train_forecast['yhat']),squared=False)
# # RMSE = mean_squared_error(X_val_df['y'],abs(train_forecast['yhat']),squared=False)
# print('RMSE------------------------------------',RMSE)

RMSE------------------------------------ 285.53500560104675


In [64]:
train_model.params

{'k': array([[0.22183961]]),
 'm': array([[0.48156699]]),
 'delta': array([[0.]]),
 'sigma_obs': array([[0.05523388]]),
 'beta': array([[-0.00035425, -0.01170126, -0.02662929,  0.06267693, -0.00441886,
          0.00502218,  0.00618214,  0.00548451,  0.00011158,  0.00107078,
          0.00514058,  0.0081017 ,  0.00080186, -0.00079351, -0.00853883,
          0.00323279,  0.00367035,  0.0028986 , -0.00408867, -0.00819345,
          0.0156409 ,  0.02234658, -0.01181101, -0.00176631, -0.00089397,
         -0.00277123]]),
 'trend': array([[0.48156699, 0.48164453, 0.48172207, ..., 0.70325152, 0.70332906,
         0.7034066 ]])}

In [71]:
from fbprophet.serialize import model_to_json, model_from_json

with open('fbprophet_hourwise.json', 'w') as fout:
    fout.write(model_to_json(train_model))  # Save model

with open('fbprophet_hourwise.json', 'r') as fin:
    temp_model = model_from_json(fin.read())  # Load model

In [69]:
preds_df = pd.DataFrame(preds)
preds_df.columns = ['preds']
preds_df.to_csv('preds_prophet_hourwise.csv',index=False)

In [70]:
preds_df.head()

,preds
0,1987.228559
1,1851.423699
2,1755.829780
3,1696.378063
4,1658.551110


In [ ]:
# train_forecast[['yhat']].to_csv('preds.csv',index=False)

In [ ]:
# train_forecast['yhat']